## Petals to the Metal - Flower Classification 
In this Getting Started competition (what is a Getting Started competition?), we're classifying 104 types of flowers based on their images drawn from five different public datasets. Some classes are very narrow, containing only a particular sub-type of flower (e.g. pink primroses) while other classes contain many sub-types (e.g. wild roses).

The dataset contains imperfections - images of flowers in odd places, or as a backdrop to modern machinery - but that's part of the challenge! Build a classifier than can see past all that, to the flowers at the heart of the images.

Files
This competition provides its files in TFRecord format. The TFRecord format is a container format frequently used in Tensorflow to group and shard data data files for optimal training performace.
Each file contains the id, label (the class of the sample, for training data) and img (the actual pixels in array form) information for many images.
Please see our Getting Started notebook or our Learn exercise for notes on how to load and use them! Additional information is available in the TPU documentation.

train/*.tfrec - training samples, including labels.

val/*.tfrec - pre-split training samples w/ labels intended to help with checking your model's performance on TPU. The split was stratified across labels.

test/*.tfrec - samples without labels - you'll be predicting what classes of flowers these fall into.

sample_submission.csv - a sample submission file in the correct format

id - a unique ID for each sample.

label - (in training data) the class of flower represented by the sample

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tpu-getting-started/sample_submission.csv
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/09-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/14-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/01-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/13-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/00-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/07-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/12-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/11-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/05-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/04-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/15-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecord

## Import Tools

In [5]:
!pip install tfrecord

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00


In [6]:
import tfrecord
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [7]:
import tensorflow as tf
import tensorflow_hub as hub

In [8]:
from kaggle_datasets import KaggleDatasets
 # you can list the bucket with "!gsutil ls $GCS_DS_PATH"

In [9]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

In [10]:
import tensorflow as tf
import math, re, os
import numpy as np

# import numpy as np

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.12.0


In [11]:
GCS_DS_PATH  = '/kaggle/input/tpu-getting-started'

In [12]:
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-224x224'

In [13]:
# Import TensorFlow into colab
import tensorflow as tf
import tensorflow_hub as hub

print("TF version :", tf.__version__)
print("TF version :", hub.__version__)

# Check for GPU availability
print("GPU" , "available (YES!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

TF version : 2.12.0
TF version : 0.12.0
GPU available (YES!!!)


## Loading Data and convert it into Image and making CSV File of image_path and Labels

In [14]:
import tensorflow as tf
import os
import cv2
import pandas as pd

# Function to decode TFRecord to image and label
def decode_tfrecord(record):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(record, LABELED_TFREC_FORMAT)  # Fix indentation here
    image = tf.image.decode_jpeg(example['image'], channels=3)
    label = tf.cast(example['class'], tf.int32)  # Fix label key here
    return image, label

In [15]:
# Set the paths for input TFRecord files and output folder
input_tfrecord_dir = '/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/train'
output_image_dir = '/kaggle/working/Train_output'
csv_filename = '/kaggle/working/train.csv'

os.makedirs(output_image_dir, exist_ok=True)

# Create a CSV file to store image names and labels
csv_data = {'Image_Name': [], 'Label': []}

# Loop through the input TFRecord files
for filename in os.listdir(input_tfrecord_dir):
    input_file = os.path.join(input_tfrecord_dir, filename)

    # Create a TFRecord dataset from the file
    dataset = tf.data.TFRecordDataset(input_file)
    # print(type(dataset))
    # break
    # dataset = dataset.repeat()

    # Iterate through the records and decode them
    # count = 0
    for record in dataset:

        # count += 1
        image, label = decode_tfrecord(record)

        # Save the image to the output folder
        image_filename = os.path.join(output_image_dir, f"{len(csv_data['Image_Name'])}.jpg")
        cv2.imwrite(image_filename, cv2.cvtColor(image.numpy(), cv2.COLOR_RGB2BGR))

        # Record the image name and label
        csv_data['Image_Name'].append(os.path.basename(image_filename))
        csv_data['Label'].append(label.numpy())

# Create a DataFrame and save it as a CSV file
df = pd.DataFrame(csv_data)
df.to_csv(csv_filename, index=False)

print("Images saved and CSV file created.")

Images saved and CSV file created.


In [16]:
# Set the paths for input TFRecord files and output folder
input_tfrecord_dir = '/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val'
output_image_dir = '/kaggle/working/val_output'
csv_filename = '/kaggle/working/val.csv'

os.makedirs(output_image_dir, exist_ok=True)

# Create a CSV file to store image names and labels
csv_data = {'Image_Name': [], 'Label': []}

# Loop through the input TFRecord files
for filename in os.listdir(input_tfrecord_dir):
    input_file = os.path.join(input_tfrecord_dir, filename)

    # Create a TFRecord dataset from the file
    dataset = tf.data.TFRecordDataset(input_file)
    # print(type(dataset))
    # break
    # dataset = dataset.repeat()

    # Iterate through the records and decode them
    # count = 0
    for record in dataset:

        # count += 1
        image, label = decode_tfrecord(record)

        # Save the image to the output folder
        image_filename = os.path.join(output_image_dir, f"aug{len(csv_data['Image_Name'])}.jpg")
        cv2.imwrite(image_filename, cv2.cvtColor(image.numpy(), cv2.COLOR_RGB2BGR))

        # Record the image name and label
        csv_data['Image_Name'].append(os.path.basename(image_filename))
        csv_data['Label'].append(label.numpy())

# Create a DataFrame and save it as a CSV file
df = pd.DataFrame(csv_data)
df.to_csv(csv_filename, index=False)

print("Images saved and CSV file created.")

Images saved and CSV file created.


## Data Augmentation

In [17]:
import pandas as pd
import os
import cv2
from tqdm import tqdm  # Optional for progress tracking

# Define the path to the input CSV file and the output directory
csv_file_path = "/kaggle/working/train.csv"
input_image_folder = "/kaggle/working/Train_output/"  # Update this to the actual image folder path
output_directory = "/kaggle/working/augmented_images"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Load the CSV data into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Function to perform image augmentation and save to the output directory


def augment_and_save(image_name, label, unique_id, brightness_factor=1.0):
    image_path = os.path.join(input_image_folder, image_name)
    image = cv2.imread(image_path)
    
    # Perform image augmentation operations here
    # Example: Rotate the image by 90 degrees
    augmented_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    
    # Adjust brightness
    augmented_image = np.clip(augmented_image * brightness_factor, 0, 255).astype(np.uint8)
    
    # Create a new filename for the augmented image
    augmented_filename = f"augment_{unique_id}_{image_name}"
    augmented_image_path = os.path.join(output_directory, augmented_filename)
    
    # Save the augmented image
    cv2.imwrite(augmented_image_path, augmented_image)
    
    # Append the filename and label to a new CSV
    augmented_data.append([augmented_filename, label])

# Create a list to store augmented data
augmented_data = []

# Loop through the CSV data and apply augmentation with different brightness levels
brightness_factors = [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]  # You can adjust these values
for index, row in tqdm(df.iterrows(), total=len(df), desc="Augmenting Images"):
    image_name = row['Image_Name']
    label = row['Label']
    
    # Generate a unique ID for each image
    unique_id = index  # You can use a different method to generate IDs
    
    # Apply augmentation with different brightness levels
    for brightness_factor in brightness_factors:
        augment_and_save(image_name, label, unique_id, brightness_factor)

# Create a new DataFrame for the augmented data
augmented_df = pd.DataFrame(augmented_data, columns=['Image_Name', 'Label'])

# Save the augmented DataFrame to a new CSV file
augmented_csv_file = "/kaggle/working/augmented_data.csv"
augmented_df.to_csv(augmented_csv_file, index=False)

print("Augmentation and CSV creation complete.")

Augmenting Images: 100%|██████████| 12753/12753 [04:49<00:00, 44.00it/s]


Augmentation and CSV creation complete.


In [19]:
# import numpy as np

# # Function to perform image augmentation (including brightness) and save to the output directory
# def augment_and_save(image_name, label, unique_id, brightness_factor=1.0):
#     image_path = os.path.join(input_image_folder, image_name)
#     image = cv2.imread(image_path)
    
#     # Perform image augmentation operations here
#     # Example: Rotate the image by 90 degrees
#     augmented_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    
#     # Adjust brightness
#     augmented_image = np.clip(augmented_image * brightness_factor, 0, 255).astype(np.uint8)
    
#     # Create a new filename for the augmented image
#     augmented_filename = f"augment_{unique_id}_{image_name}"
#     augmented_image_path = os.path.join(output_directory, augmented_filename)
    
#     # Save the augmented image
#     cv2.imwrite(augmented_image_path, augmented_image)
    
#     # Append the filename and label to a new CSV
#     augmented_data.append([augmented_filename, label])

# # Create a list to store augmented data
# augmented_data = []

# # Loop through the CSV data and apply augmentation with different brightness levels
# brightness_factors = [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]  # You can adjust these values
# for index, row in tqdm(df.iterrows(), total=len(df), desc="Augmenting Images"):
#     image_name = row['Image_Name']
#     label = row['Label']
    
#     # Generate a unique ID for each image
#     unique_id = index  # You can use a different method to generate IDs
    
#     # Apply augmentation with different brightness levels
#     for brightness_factor in brightness_factors:
#         augment_and_save(image_name, label, unique_id, brightness_factor)

# # Create a new DataFrame for the augmented data
# augmented_df = pd.DataFrame(augmented_data, columns=['Image_Name', 'Label'])

# # Save the augmented DataFrame to a new CSV file
# augmented_csv_file = "/kaggle/working/augmented_data.csv"
# augmented_df.to_csv(augmented_csv_file, index=False)

# print("Augmentation and CSV creation complete.")

In [20]:
import pandas as pd
df1 = pd.read_csv("/kaggle/working/train.csv")
df2 = pd.read_csv("/kaggle/working/val.csv") 
df3 = pd.read_csv("/kaggle/working/augmented_data.csv") 
#z = pd.concat([df1, df2])
# labels_csv = pd.concat([t, df3])

In [21]:
df1['Image_Name']='/kaggle/working/Train_output/'+df1['Image_Name']
df2['Image_Name']='/kaggle/working/val_output/'+df2['Image_Name']
df3['Image_Name']='/kaggle/working/augmented_images/'+df3['Image_Name']

In [22]:
t = pd.concat([df1, df2],axis=0,ignore_index=True)
labels_csv = pd.concat([t, df3],axis=0,ignore_index=True)

In [23]:
labels_csv

,Image_Name,Label
0,/kaggle/working/Train_output/0.jpg,70
1,/kaggle/working/Train_output/1.jpg,67
2,/kaggle/working/Train_output/2.jpg,80
3,/kaggle/working/Train_output/3.jpg,82
4,/kaggle/working/Train_output/4.jpg,73
...,...,...
105731,/kaggle/working/augmented_images/augment_12752...,64
105732,/kaggle/working/augmented_images/augment_12752...,64
105733,/kaggle/working/augmented_images/augment_12752...,64
105734,/kaggle/working/augmented_images/augment_12752...,64


In [24]:
import pandas as pd
# labels_csv = pd.read_csv('/kaggle/working/train.csv')
print(labels_csv.describe())
print(labels_csv.head())

               Label
count  105736.000000
mean       55.611041
std        30.615215
min         0.000000
25%        32.000000
50%        56.000000
75%        78.000000
max       103.000000
                           Image_Name  Label
0  /kaggle/working/Train_output/0.jpg     70
1  /kaggle/working/Train_output/1.jpg     67
2  /kaggle/working/Train_output/2.jpg     80
3  /kaggle/working/Train_output/3.jpg     82
4  /kaggle/working/Train_output/4.jpg     73


In [25]:
labels_csv.drop_duplicates(inplace=True)

In [26]:
labels_csv.head()

,Image_Name,Label
0,/kaggle/working/Train_output/0.jpg,70
1,/kaggle/working/Train_output/1.jpg,67
2,/kaggle/working/Train_output/2.jpg,80
3,/kaggle/working/Train_output/3.jpg,82
4,/kaggle/working/Train_output/4.jpg,73


In [27]:
# Create pathnames from image ID's
filenames = [ fname for fname in labels_csv["Image_Name"]]

# Check the first 10 filenames
filenames[:10]

['/kaggle/working/Train_output/0.jpg',
 '/kaggle/working/Train_output/1.jpg',
 '/kaggle/working/Train_output/2.jpg',
 '/kaggle/working/Train_output/3.jpg',
 '/kaggle/working/Train_output/4.jpg',
 '/kaggle/working/Train_output/5.jpg',
 '/kaggle/working/Train_output/6.jpg',
 '/kaggle/working/Train_output/7.jpg',
 '/kaggle/working/Train_output/8.jpg',
 '/kaggle/working/Train_output/9.jpg']

In [28]:
import numpy as np
labels = labels_csv["Label"].to_numpy() # convert labels column to NumPy array
labels[:10]

array([70, 67, 80, 82, 73,  4, 49, 80, 77, 68])

In [29]:
# Find the unique label values
unique_labels = np.unique(labels)
len(unique_labels)

104

In [30]:
print(labels[0])
labels[0] == unique_labels # use comparison operator to create boolean array

70


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [31]:
# Turn every label into a boolean array
boolean_labels = [label == np.array(unique_labels) for label in labels]
boolean_labels[:2]

[array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False]),
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, Fal

In [32]:
# Example: Turning a boolean array into integers
print(labels[0]) # original label
print(np.where(unique_labels == labels[0])[0][0]) # index where label occurs
print(boolean_labels[0].argmax()) # index where label occurs in boolean array
print(boolean_labels[0].astype(int)) # there will be a 1 where the sample label occurs

70
70
70
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [33]:
# Setup X & y variables
X = filenames
y = boolean_labels

In [34]:
len(X)

29218

In [35]:
# Import train_test_split from Scikit-Learn
from sklearn.model_selection import train_test_split

# Split them into training and validation using NUM_IMAGES
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y,
                                                  test_size=0.17,
                                                  random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

(24250, 24250, 4968, 4968)

## Preprocessing Data

In [36]:
# Define image size
IMG_SIZE = 224

def process_image(image_path):
  """
  Takes an image file path and turns it into a Tensor.
  """
  # Read in image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the colour channel values from 0-225 values to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired size (224, 244)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

In [37]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

In [54]:
# Define the batch size, 32 is a good default
BATCH_SIZE = 64

# Create a function to turn data into batches
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (x) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle it if it's validation data.
  Also accepts test data as input (no labels).
  """
  # If the data is a test dataset, we probably don't have labels
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only filepaths
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  # If the data if a valid dataset, we don't need to shuffle it
  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                               tf.constant(y))) # labels
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

  else:
    # If the data is a training dataset, we shuffle it
    print("Creating training data batches...")
    # Turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                              tf.constant(y))) # labels

    # Shuffling pathnames and labels before mapping image processor function is faster than shuffling images
    data = data.shuffle(buffer_size=len(x))

    # Create (image, label) tuples (this also turns the image path into a preprocessed image)
    data = data.map(get_image_label)

    # Turn the data into batches
    data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [55]:
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

Creating training data batches...
Creating validation data batches...


In [56]:
train_data.element_spec, val_data.element_spec

((TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 104), dtype=tf.bool, name=None)),
 (TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 104), dtype=tf.bool, name=None)))

## Model Creating

In [57]:
# Setup input shape to the model
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # batch, height, width, colour channels

# Setup output shape of the model
OUTPUT_SHAPE = len(unique_labels) # number of unique labels

# Setup model URL from TensorFlow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5"

In [58]:
import tensorflow as tf
import tensorflow_hub as hub

# Create a function which builds a Keras model with dropout and L2 regularization
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL, dropout_rate=0.1, l2_regularization=0.001):
    print("Building model with:", MODEL_URL)

    # Setup the model layers
    model = tf.keras.Sequential([
        hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
        tf.keras.layers.Dropout(rate=dropout_rate), # Dropout layer to prevent overfitting
        tf.keras.layers.Dense(units=128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_regularization)), # Additional hidden layer with L2 regularization
        tf.keras.layers.Dropout(rate=dropout_rate), # Dropout layer
        tf.keras.layers.Dense(units=OUTPUT_SHAPE, activation="softmax") # Layer 2 (output layer)
    ])

    # Compile the model
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"]
    )

    # Build the model
    model.build(INPUT_SHAPE)

    return model


In [59]:
# Create a model and check its details
IMAGE_SIZE=[224,224,3]
model = create_model()
model.summary()

Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 1001)              5432713   
                                                                 
 dropout_8 (Dropout)         (None, 1001)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               128256    
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 104)               13416     
                                                                 
Total params: 5,574,385
Trainable params: 141,672
Non-trainable params: 5,432,713
___________

In [60]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [61]:
import datetime

# Create a function to build a TensorBoard callback
def create_tensorboard_callback():
  # Create a log directory for storing TensorBoard logs
  logdir = os.path.join("/kaggle/working",
                        # Make it so the logs get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)

In [62]:
# Create early stopping (once our model stops improving, stop training)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3) # stops after 3 rounds of no improvements

In [63]:
NUM_EPOCHS = 100

In [64]:
STEPS_PER_EPOCH = len(X_train) // 32
# Build a function to train and return a trained model
def train_model():
  """
  Trains a given model and returns the trained version.
  """
  # Create a model
  model = create_model()

  # Create new TensorBoard session everytime we train a model
  tensorboard = create_tensorboard_callback()

  # Fit the model to the data passing it the callbacks we created
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=val_data,
            validation_freq=1, # check validation metrics every epoch
            callbacks=[early_stopping],
         )

  return model

In [65]:
# Fit the model to the data
model = train_model()

Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5
Epoch 1/100
379/379 [==============================] - 34s 77ms/step - loss: 2.0185 - accuracy: 0.5713 - val_loss: 1.1648 - val_accuracy: 0.7536
Epoch 2/100
379/379 [==============================] - 28s 74ms/step - loss: 1.1265 - accuracy: 0.7517 - val_loss: 0.9894 - val_accuracy: 0.7919
Epoch 3/100
379/379 [==============================] - 28s 74ms/step - loss: 0.9453 - accuracy: 0.7953 - val_loss: 0.9405 - val_accuracy: 0.8041
Epoch 4/100
379/379 [==============================] - 28s 75ms/step - loss: 0.8520 - accuracy: 0.8176 - val_loss: 0.8935 - val_accuracy: 0.8090
Epoch 5/100
379/379 [==============================] - 29s 75ms/step - loss: 0.7891 - accuracy: 0.8360 - val_loss: 0.8730 - val_accuracy: 0.8166
Epoch 6/100
379/379 [==============================] - 28s 73ms/step - loss: 0.7443 - accuracy: 0.8478 - val_loss: 0.8495 - val_accuracy: 0.8281
Epoch 7/100
379/379 [================

## we gain 82.6 percent Accuracy